In [1]:
import os
import os.path
import glob

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
import matplotlib.pyplot as plt
import matplotlib.cm as colormap

import numpy as np
np.random.seed(1)

from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications import VGG16
from keras.applications.imagenet_utils import preprocess_input
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,Flatten,Dense,Dropout,GlobalAveragePooling2D,Conv2D,MaxPooling2D

Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyImagesWidth"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
# Encoding classes (y) into integers (y_encoded) and then generating one-hot-encoding (Y)
encoder = LabelEncoder()
encoder.fit(y)
y_encoded = encoder.transform(y)
Y = np_utils.to_categorical(y_encoded)

In [5]:
# Creating base_model (VGG16 convolutional layers)
image_shape = (224, 224, 3)                                                                                                                                               
base_model = VGG16(weights='imagenet', input_shape=image_shape, include_top=False)         

In [6]:
filename = 'virussign-vgg16features.npy'
if os.path.exists(filename):
    print("Loading VGG16 extracted features from %s ..." %(filename))
    vggfeatures = np.load(filename)
else:
    print("Extracting features from VGG16 convolutional layers ...")
    vggfeatures = base_model.predict(X)
    print("Saving VGG16 extracted features into %s ..." %(filename))
    np.save(filename, vggfeatures)

Loading VGG16 extracted features from virussign-vgg16features.npy ...


In [7]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)
skfind = [None] * kfold  # skfind[i][0] -> train indices, skfind[i][1] -> test indices
cnt = 0                                              
for index in skf.split(X, y):         
    skfind[cnt] = index                                                 
    cnt += 1 

In [8]:
# Training top_model and saving min training loss weights
num_epochs=1000
history = []
conf_mat = np.zeros((len(list_fams),len(list_fams))) # Initializing the Confusion Matrix
checkpointer = ModelCheckpoint(filepath='model-min_loss-vgg16-2layers-160neurons-relu-0.4dropout-Adam-1000epochs.h5', monitor='loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
for i in range(kfold):
    train_indices = skfind[i][0]
    test_indices = skfind[i][1]
    X_train = vggfeatures[train_indices]
    Y_train = Y[train_indices]
    X_test = vggfeatures[test_indices]
    Y_test = Y[test_indices]
    y_test = y[test_indices]

    top_model = Sequential() 
    top_model.add(Flatten(input_shape=base_model.output_shape[1:])) # input_shape=(7,7,512)                                                
    top_model.add(Dense(160, activation='relu', name='fc1'))
    top_model.add(Dropout(0.4))                
    top_model.add(Dense(160, activation='relu', name='fc2'))
    top_model.add(Dropout(0.4))              
    top_model.add(Dense(num_classes, activation='softmax', name='predictions'))                             
    top_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    h = top_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=num_epochs, batch_size=X_train.shape[0], verbose=1, callbacks=[checkpointer])
    history.append(h)
    
    y_prob = top_model.predict(X_test, verbose=0)  # Testing
    y_pred = np.argmax(y_prob, axis=1)
    print("[%d] Test acurracy: %.4f" %(i,accuracy_score(y_test,y_pred)))
    cm = confusion_matrix(y_test,y_pred)  # Compute confusion matrix for this fold
    conf_mat = conf_mat + cm  # Compute global confusion matrix

Train on 9112 samples, validate on 1024 samples
Epoch 1/1000
9112/9112 [==============================] - 31s - loss: 12.8900 - acc: 0.0588 - val_loss: 6.7694 - val_acc: 0.4570
Epoch 2/1000
9112/9112 [==============================] - 7s - loss: 9.1721 - acc: 0.3066 - val_loss: 5.1364 - val_acc: 0.5479
Epoch 3/1000
9112/9112 [==============================] - 6s - loss: 7.2253 - acc: 0.4316 - val_loss: 3.8872 - val_acc: 0.6299
Epoch 4/1000
9112/9112 [==============================] - 6s - loss: 5.8564 - acc: 0.4915 - val_loss: 2.3118 - val_acc: 0.6797
Epoch 5/1000
9112/9112 [==============================] - 6s - loss: 4.4228 - acc: 0.5313 - val_loss: 1.5118 - val_acc: 0.7188
Epoch 6/1000
9112/9112 [==============================] - 6s - loss: 3.2894 - acc: 0.5262 - val_loss: 1.2566 - val_acc: 0.7217
Epoch 7/1000
9112/9112 [==============================] - 6s - loss: 2.3900 - acc: 0.5190 - val_loss: 1.3017 - val_acc: 0.6670
Epoch 8/1000
9112/9112 [==============================] - 6s 

9112/9112 [==============================] - 6s - loss: 0.7083 - acc: 0.7906 - val_loss: 0.6105 - val_acc: 0.8477
Epoch 65/1000
9112/9112 [==============================] - 6s - loss: 0.6971 - acc: 0.7916 - val_loss: 0.6129 - val_acc: 0.8477
Epoch 66/1000
9112/9112 [==============================] - 6s - loss: 0.6832 - acc: 0.7984 - val_loss: 0.6257 - val_acc: 0.8477
Epoch 67/1000
9112/9112 [==============================] - 6s - loss: 0.6811 - acc: 0.7987 - val_loss: 0.6260 - val_acc: 0.8486
Epoch 68/1000
9112/9112 [==============================] - 6s - loss: 0.6704 - acc: 0.7976 - val_loss: 0.6245 - val_acc: 0.8496
Epoch 69/1000
9112/9112 [==============================] - 5s - loss: 0.6743 - acc: 0.7993 - val_loss: 0.6174 - val_acc: 0.8496
Epoch 70/1000
9112/9112 [==============================] - 6s - loss: 0.6803 - acc: 0.7964 - val_loss: 0.6098 - val_acc: 0.8506
Epoch 71/1000
9112/9112 [==============================] - 6s - loss: 0.6682 - acc: 0.7992 - val_loss: 0.6156 - val_ac

9112/9112 [==============================] - 6s - loss: 0.5593 - acc: 0.8232 - val_loss: 0.5987 - val_acc: 0.8623
Epoch 128/1000
9112/9112 [==============================] - 6s - loss: 0.5533 - acc: 0.8231 - val_loss: 0.5985 - val_acc: 0.8652
Epoch 129/1000
9112/9112 [==============================] - 6s - loss: 0.5532 - acc: 0.8201 - val_loss: 0.6057 - val_acc: 0.8682
Epoch 130/1000
9112/9112 [==============================] - 5s - loss: 0.5498 - acc: 0.8228 - val_loss: 0.6053 - val_acc: 0.8682
Epoch 131/1000
9112/9112 [==============================] - 6s - loss: 0.5383 - acc: 0.8295 - val_loss: 0.6175 - val_acc: 0.8623
Epoch 132/1000
9112/9112 [==============================] - 5s - loss: 0.5365 - acc: 0.8251 - val_loss: 0.6237 - val_acc: 0.8643
Epoch 133/1000
9112/9112 [==============================] - 6s - loss: 0.5467 - acc: 0.8231 - val_loss: 0.6108 - val_acc: 0.8682
Epoch 134/1000
9112/9112 [==============================] - 6s - loss: 0.5453 - acc: 0.8231 - val_loss: 0.6054 -

9112/9112 [==============================] - 5s - loss: 0.5061 - acc: 0.8330 - val_loss: 0.6522 - val_acc: 0.8643
Epoch 191/1000
9112/9112 [==============================] - 6s - loss: 0.5170 - acc: 0.8321 - val_loss: 0.6443 - val_acc: 0.8662
Epoch 192/1000
9112/9112 [==============================] - 6s - loss: 0.4995 - acc: 0.8359 - val_loss: 0.6339 - val_acc: 0.8662
Epoch 193/1000
9112/9112 [==============================] - 6s - loss: 0.4924 - acc: 0.8386 - val_loss: 0.6307 - val_acc: 0.8662
Epoch 194/1000
9112/9112 [==============================] - 6s - loss: 0.5131 - acc: 0.8333 - val_loss: 0.6384 - val_acc: 0.8691
Epoch 195/1000
9112/9112 [==============================] - 6s - loss: 0.4978 - acc: 0.8357 - val_loss: 0.6445 - val_acc: 0.8652
Epoch 196/1000
9112/9112 [==============================] - 6s - loss: 0.5045 - acc: 0.8346 - val_loss: 0.6260 - val_acc: 0.8682
Epoch 197/1000
9112/9112 [==============================] - 6s - loss: 0.5113 - acc: 0.8311 - val_loss: 0.6147 -

KeyboardInterrupt: 

In [ ]:
# Computing the average accuracy
avg_acc = np.trace(conf_mat)/np.sum(conf_mat)
print("Average acurracy: %.4f" %(avg_acc))

In [ ]:
def plot_acc(history):
    figure = plt.gcf()
    figure.set_size_inches(24, 9)
    ax = plt.subplot()
    plt.title('Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    colors = iter(colormap.gist_rainbow(np.linspace(0, 1, len(history))))
    for i in range(len(history)):
        color=next(colors)
        plt.plot(history[i].history['acc'], label='Train '+str(i), color=color, linestyle = 'solid')
        plt.plot(history[i].history['val_acc'], label='Test '+str(i), color=color, linestyle = 'dotted')
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,0.0,1.0))
    plt.legend()
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True)
    plt.show()

In [ ]:
def plot_loss(history):
    figure = plt.gcf()
    figure.set_size_inches(24, 9)
    ax = plt.subplot()
    plt.title('Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    colors = iter(colormap.gist_rainbow(np.linspace(0, 1, len(history))))
    for i in range(len(history)):
        color=next(colors)
        plt.plot(history[i].history['loss'], label='Train '+str(i), color=color, linestyle = 'solid')
        plt.plot(history[i].history['val_loss'], label='Test '+str(i), color=color, linestyle = 'dotted')
    plt.legend()
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True)
    plt.show()

In [ ]:
plot_acc(history)

In [ ]:
plot_loss(history)

In [ ]:
print("Plotting the confusion matrix")
figure = plt.gcf()
figure.set_size_inches(18, 12)
plt.imshow(conf_mat,interpolation='nearest',cmap='coolwarm')
for row in range(len(list_fams)):
    for col in range(len(list_fams)):
        plt.annotate(str(int(conf_mat[row][col])),xy=(col,row),ha='center',va='center')
plt.xticks(range(len(list_fams)),list_fams,rotation=90,fontsize=10)
plt.yticks(range(len(list_fams)),list_fams,fontsize=10)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.colorbar()
plt.show()

In [ ]:
print("Plotting the confusion matrix normalized")
conf_mat_norm = conf_mat/np.sum(conf_mat,axis=1)  # Normalizing the confusion matrix
conf_mat_norm = np.around(conf_mat_norm,decimals=2)  # rounding to display in figure
figure = plt.gcf()
figure.set_size_inches(20, 12)
plt.imshow(conf_mat_norm,interpolation='nearest',cmap='seismic')
for row in range(len(list_fams)):
    for col in range(len(list_fams)):
        plt.annotate(str(conf_mat_norm[row][col]),xy=(col,row),ha='center',va='center')
plt.xticks(range(len(list_fams)),list_fams,rotation=90,fontsize=10)
plt.yticks(range(len(list_fams)),list_fams,fontsize=10)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.colorbar()
plt.show()

In [ ]:
print("Plotting the confusion matrix normalized")
conf_mat_norm = conf_mat/np.sum(conf_mat,axis=1)  # Normalizing the confusion matrix
conf_mat_norm = np.around(conf_mat_norm,decimals=2)  # rounding to display in figure
figure = plt.gcf()
figure.set_size_inches(20, 12)
plt.imshow(conf_mat_norm,interpolation='nearest',cmap='binary')
#for row in range(len(list_fams)):
#    for col in range(len(list_fams)):
#        plt.annotate(str(conf_mat_norm[row][col]),xy=(col,row),ha='center',va='center')
plt.xticks(range(len(list_fams)),list_fams,rotation=90,fontsize=10)
plt.yticks(range(len(list_fams)),list_fams,fontsize=10)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.colorbar()
plt.show()